In [4]:
# get articles from elastic search

In [2]:
from elasticsearch import Elasticsearch
import json
from elasticsearch.helpers import scan

ModuleNotFoundError: No module named 'elasticsearch'

In [6]:
es = Elasticsearch([{'host':'localhost', 'port':9200}], http_auth=('uchicago', '3fMy7Wq90LDWBCMEXt6j'))

In [7]:
res = scan(
    es,
    index='credibility_training_data', 
    size=5000, 
    query={'query':{'match_all':{}}}, 
)

res_dict = {}
counter = 0
for each in res:
    uid = each['_id']
    url = each['_source']['link']
    credibility = each['_source']["credibility"]
    domain = each['_source']["registered_domain"]
    try:
        text = each['_source']["text"]
        source = each['_source']['source']
    except:
        continue
    res_dict[uid]={"url": url, "source": source, "credibility": credibility, "domain": domain, "text": text}
    if counter == 0:
        print(each)
        counter += 1

print(len(res_dict))
        
#     # save info to a json file
with open('data.json', 'w') as f:
    json.dump(res_dict, f)



{'_index': 'credibility_training_data', '_type': '_doc', '_id': '204f5d99-c9de-4bce-ae2f-5e98085b4930', '_score': None, '_source': {'link': 'https://gazette.com/election-coverage/q-a-s-with-the-colorado-gubernatorial-candidates/article_37967414-d940-11e8-986a-972c594bffd9.html', 'source': 'gazette', 'language': 'en', 'credibility': 'medium', 'registered_domain': 'gazette.com', 'text': 'Here are answers to questions we sent to all four major- and minor-party governor candidates in November’s election.\n\nBill Hammons, Unity Party\n\nLt. Gov. running mate: Eric Bodenstab\n\nWhat makes you a better choice than your opponents?\n\nI’m the best choice for the simple reason that I’m truly something new under the sun (for once). I founded (with my father) the now-38-state Unity Party of America the day after the 2004 Election, and founded the Unity Party of Colorado by personally gathering well over 1,000 valid voter signatures in multiple statewide races, placing myself as a Unity Party candi

In [8]:

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import numpy as np
import scipy.stats


In [9]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

articles_attr = {}
subjectivity_vals = np.empty(shape=[0, 1])
polarity_vals = np.empty(shape=[0, 1])
cred_score = np.empty(shape=[0, 1])
id_legend = np.empty(shape=[0, 1])
publishers_vals = np.empty(shape=[0, 1])
publisher_hashes = {}

# subjectivity_vals_low = np.empty(shape=[0, 1])
# polarity_vals_low = np.empty(shape=[0, 1])
# subjectivity_vals_med = np.empty(shape=[0, 1])
# polarity_vals_med = np.empty(shape=[0, 1])
# subjectivity_vals_high = np.empty(shape=[0, 1])
# polarity_vals_high = np.empty(shape=[0, 1])
# polarity_vals_high = np.empty(shape=[0, 1])

labeled_scores = {}
counter = 0

for key, value in res_dict.items():
    doc = nlp(value["text"])
    articles_attr[key] = {"polarity": doc._.polarity, "subjectivity": doc._.subjectivity, "assessments": doc._.assessments, "source": value["source"]}
    subjectivity_vals = np.append(subjectivity_vals, doc._.subjectivity)
    polarity_vals = np.append(polarity_vals, doc._.polarity)
    id_legend = np.append(id_legend, key)
    credi_score = value["credibility"]
    cred_score = np.append(cred_score, credi_score)
    labeled_scores[key]= credi_score
    source_h = None
    try:
        source_h = publisher_hashes[value["source"]]
    except KeyError:
        publisher_hashes[value["source"]] = counter
        source_h = counter
        counter += 1
    finally:
        publishers_vals = np.append(publishers_vals, source_h)
        

In [15]:
#excluding outliers
subjectivity_mean = np.mean(subjectivity_vals)
subjectivity_sd = np.std(subjectivity_vals)
polarity_mean = np.mean(polarity_vals)
polarity_sd = np.std(polarity_vals)

id_legend_save = np.copy(id_legend)

excluded = []
excluded_index = []

upper_subjectivity = subjectivity_mean + 3*subjectivity_sd
lower_subjectivity = subjectivity_mean - 3*subjectivity_sd
upper_polarity = polarity_mean + 3*polarity_sd
lower_polarity = polarity_mean - 3*polarity_sd

for i in range(len(id_legend_save)):
    if (subjectivity_vals[i] > upper_subjectivity) or (subjectivity_vals[i] < lower_subjectivity) or (polarity_vals[i] > upper_polarity) or (polarity_vals[i] < lower_polarity):
        excluded.append(id_legend_save[i])
        excluded_index.append(i)
        
subjectivity_vals = np.delete(subjectivity_vals, excluded_index)
polarity_vals = np.delete(polarity_vals, excluded_index)
id_legend = np.delete(id_legend, excluded_index)
cred_score = np.delete(cred_score, excluded_index)

#write out the vectors, excluded or no?
with open('vectors.txt', 'w') as f:
    for i in range(len(cred_score)):
        f.write("id: ")
        f.write(id_legend[i].astype('str'))
        f.write("\n")
        f.write("polarity: ")
        f.write(polarity_vals[i].astype('str'))
        f.write("\n")
        f.write("subjectivity: ")
        f.write(subjectivity_vals[i].astype('str'))
        f.write("\n")
        f.write("cred_score: ")
        f.write(cred_score[i].astype('str'))
        f.write("\n")
        f.write("publisher_val: ")
        f.write(publishers_vals[i].astype('str')])
        f.write(str(publisher_hashes[res_dict[id_legend[i]]["source"]]))
        f.write("\n")
        f.write("publisher: ")
        f.write(res_dict[id_legend[i]]["source"])
        f.write()
        f.write("\n")
        f.write("-------------\n")
        #add publisher to above and here
    

print("number of excluded: ", len(excluded_index))
print("number of included: ", len(id_legend))

with open('publisher_hash.txt', 'w') as f:
    for key, value in publisher_hashes.items():
        f.write("Publisher: ")
        f.write(str(key))
        f.write(" , hashed: ")
        f.write(str(value))
        f.write("\n")

number of excluded:  59
number of included:  38867


In [ ]:
import matplotlib.pyplot as plt
import matplotlib

def do_plot(subjectivity_vals, polarity_vals, tags, cred_score):
    # Plot
#     colors = ["red", "green", "blue"]
    cdict = {"high": 'red', "medium": 'blue', "low": 'green'}
#     fig = plt.figure(figsize=(8,8))
    fig, ax = plt.subplots(figsize=(20,20))
    for g in np.unique(cred_score):
        ix = np.where(cred_score == g)
        ax.scatter(subjectivity_vals[ix], polarity_vals[ix], c = cdict[g], label = g)
    ax.legend()
#     plt.scatter(subjectivity_vals, polarity_vals, c = cdict[g])
    plt.xlabel("Subjectivity")
    plt.ylabel("Polarity")
    plt.title("Subjectivity-Polarity Word-level")
    plt.legend()
    m, b = np.polyfit(subjectivity_vals, polarity_vals, 1) #regression
    plt.plot(subjectivity_vals, m*subjectivity_vals + b)
    plt.savefig("initial_plot.png")
    plt.close()

do_plot(subjectivity_vals, polarity_vals, id_legend, cred_score)

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(subjectivity_vals,  polarity_vals)
with open('stats.txt', 'w') as f:
    f.write("slope: ")
    f.write(slope.astype('str'))
    f.write("\n")
    f.write("intercept: ")
    f.write(intercept.astype('str'))
    f.write("\n")
    f.write("r_value: ")
    f.write(r_value.astype('str'))
    f.write("\n")
    f.write("p_value: ")
    f.write(p_value.astype('str'))
    f.write("\n")
    f.write("std_err: ")
    f.write(std_err.astype('str'))
    f.write("-------------\n")

In [ ]:
# # print(res.to_dict)
# counter = 0
# for each in res:
#     if counter == 3:
#         break
#     print(each)
#     counter += 1


# results = []

# res = es.search(
#     index='credibility_training_data', 
#     scroll = '2m',
#     search_type = 'scan',
#     size=5000, 
#     body={'query':{'match_all':{}}}, 
# )

# # get and return a list of urls
# def get_urls():
#     results = res['hits']['hits']
#     urls = {}
#     for u in results:
#         url_id = u['_id']
#         url = u['_source']['link']
#         urls[url_id] = url
#     return urls
    
# urls = get_urls()
# len(urls)

# res = es.search(
#     index='credibility_training_data', 
#     size=5000, 
#     body={'query':{'match_all':{}}}, 
#     _source=['source', 'link']
# )


# results = res['hits']['hits']
# print(len(results))
#     counter += 1
    
# uid = results[0]['_id']
# url = results[0]['_source']['link']
# source = results[0]['_source']['source']
# credibility = results[0]['_source']["credibility"]
# domain = results[0]['_source']["registered_domain"]
# text = results[0]['_source']["text"]
# # print(uid)
# # print(url)
# print(source)
# print(credibility)
# print(domain)
# print(text)

# len_returned = 5000
# results = []
# while len_returned == 5000:
#     res = es.search(
#     index='emm_data', 
#     size=5000, 
#     body={'query':{'match_all':{}}}, 
#     _source=['source', 'link']
#     )
#     received = res['hits']['hits']
#     results.extend(received)
#     len_returned = len(received)

# import json

# # store article's id and url in json
# def get_id_and_url(res):
#     results = res['hits']['hits']
# #     print(results)
#     res_dict = {}
#     urls = {}
#     for r in results:
# #         print(r)
#         uid = r['_id']
#         url = r['_source']['link']
# #         source = r['_source']['source']
#         credibility = r['_source']["credibility"]
#         domain = r['_source']["registered_domain"]
# #         text = r['_source']["text"]
# #         res_dict[uid]={"url": url, "source": source, "credibility": credibility, "domain": domain, "text": text}
        
#     # save info to a json file
#     with open('data.json', 'w') as f:
#         json.dump(res_dict, f)
    
#     return urls

# print(res)
# # urls = get_id_and_url(res)

# !pip install newspaper3k

In [ ]:
# Extracting information from the article

In [ ]:
# import newspaper
# from newspaper import Article

In [ ]:
# # get article's url
# url = res['hits']['hits'][0].get('_source')['link']

# # create Article object
# article = Article(url)

# # download & parse the article
# article.download()
# article.parse()

# # extract NL properties from the text
# article.nlp()

In [ ]:
# get author(s), publish date & top image
f'author: {article.authors},    publication date: {article.publish_date},    top image: {article.top_image}'

In [ ]:
# # get the article's text
# article.text

In [ ]:
# now perform NLP on the article's text with spaCy

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
# store the article's text
# article_text = article.text

# # tokenize the text
# article_doc = nlp(article_text)

In [ ]:
# # extract tokens from the doc
# article_tokens = [token.text for token in article_doc]

# # print the tokens
# article_tokens

In [ ]:
# # extract info and perfom NLP on articles
# def extract_article_info():
#     articles_dict = {}
# #     counter = 0
#     for url_id, url in urls.items():
# #         if counter == 100:
# #             break
# #         else:
# #             article = Article(url)
# #             if article.is_valid_url():
# #                 article.download()
# #                 article.parse()
# #                 articles_dict[url_id] = article.text
        
#         article = Article(url)
#         if article.is_valid_url():
#             article.download()
#         try:    
#             article.parse()
#         except:
#             continue
#         articles_dict[url_id] = article.text
# #         counter+=1
    
#     return articles_dict

# articles_dict = extract_article_info()

In [ ]:
# print(extract_article_info().items())

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import numpy as np
import scipy.stats


In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

articles_attr = {}
subjectivity_vals = np.empty(shape=[0, 1])
polarity_vals = np.empty(shape=[0, 1])
id_legend = np.empty(shape=[0, 1])

for key, value in articles_dict.items():
    doc = nlp(value)
    articles_attr[key] = [doc._.polarity, doc._.subjectivity, doc._.assessments]
    subjectivity_vals = np.append(subjectivity_vals, doc._.subjectivity)
    polarity_vals = np.append(polarity_vals, doc._.polarity)
    id_legend = np.append(id_legend, key)

subjectivity_mean = np.mean(subjectivity_vals)
subjectivity_sd = np.std(subjectivity_vals)
polarity_mean = np.mean(polarity_vals)
polarity_sd = np.std(polarity_vals)

id_legend_save = np.copy(id_legend)

excluded = []
excluded_index = []

for i in range(len(id_legend_save)):
    if (subjectivity_vals[i] > subjectivity_mean + 2.5*subjectivity_sd) or (subjectivity_vals[i] < subjectivity_mean - 2.5*subjectivity_sd) or (polarity_vals[i] > polarity_mean + 2.5*polarity_sd) or (polarity_vals[i] < polarity_mean - 2.5*polarity_sd):
        excluded.append(id_legend_save[i])
        excluded_index.append(i)
        
subjectivity_vals = np.delete(subjectivity_vals, excluded_index)
polarity_vals = np.delete(polarity_vals, excluded_index)
id_legend = np.delete(id_legend, excluded_index)

    
#subjectivity: 0.0 = objective, 1.0 = subjective
#polarity: measures level of approval/disapproval
#assessments = list of polarity, subjectivity scores for assessed tokens


#x-y plot of subjectivity, polarity
#see if there is any relationship, clustering

import matplotlib.pyplot as plt

def do_plot(subjectivity_vals, polarity_vals, tags):
    # Plot
    plt.scatter(subjectivity_vals, polarity_vals, label="oridinal data")
    plt.xlabel("Subjectivity")
    plt.ylabel("Polarity")
    plt.title("Subjectivity-Polarity Word-level")
    plt.legend()
    m, b = np.polyfit(subjectivity_vals, polarity_vals, 1) #regression
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(subjectivity_vals,  polarity_vals)
    with open('stats.txt', 'w') as f:
        f.write("slope: ", slope)
        f.write("intercept: ", intercept)
        f.write("r_value: ", r_value)
        f.write("p_value: ", p_value)
        f.write("std_err: ", std_err)
    plt.plot(subjectivity_vals, m*subjectivity_vals + b)
    plt.savefig("initial_plot.png")
    plt.close()

do_plot(subjectivity_vals, polarity_vals, id_legend)

    